# Introduction

First, you need to grab your annotations and assemble a training corpus.

In our work scenario, we use `Webanno` to annotate and store the manually corrected texts. We keep the gold standards in `data/IOB`.

The files are 1 per page of the volume. The filenames *must* have a sequential number identifying the sequence in which we have annotated them. Every file in the `data/IOB` folder is structured as in this example:

```csv
Uebrigens	NN	<unknown>	_	O	_
bin	VAFIN	sein	_	O	_
ich	PPER	ich	_	O	_
etwas	ADV	etwas	_	O	_
zu	PTKA	zu	_	O	_
sehr	ADV	sehr	_	O	_
zersplittert	VVPP	zersplittern	_	O	_
```

The columns are:
* form
* pos (TreeTagger German tagset)
* lemma
* textual layer annotation (not used, but must be there)
* NER (the actual column we're interested in)
* NE id to gazetteers (not used, but must be there)

Unfortunately, this output is not natively produced by Webanno, and the full export of the project yields a lot of errors, with no informative log. We'll update the `data/IOB` folder manually with the last volumes.

First, we need to make sure that we have access to a couple of dependency projects that I am running locally are available to $PYTHONPATH. The deps are:
* `pywebanno`: see [here](https://github.com/dainst/PyWebanno)
* my `custom-corpus-readers` (see [here](https://github.com/francescomambrini/custom-corpus-readers)) for NLTK, which include support for flexible TSV files.

In [2]:
#this will work for my machine; install the dependency properly
import sys
sys.path.append("../../PyWebanno/pywebanno")
sys.path.append("../../custom-corpus-readers")
sys.path.append("../")

Now these imports should work

In [3]:
import pywebanno as pywa
from corpusreader import WebAnnoTsvCorpusReader
from korr_corpusreader import KorrIOBCorpusReader

## Methodology

"Conditional Random Field" ([CRF](https://en.wikipedia.org/wiki/Conditional_random_field)) is an algorithm of machine learning that is very often used in Named Entity Recognition ([NER](https://en.wikipedia.org/wiki/Named-entity_recognition)) tasks. It is suitable to treat linguistic texts where the context of a token (i.e. the series of features of the preceding and following tokens) is often very important to guess the right tag.

See [here](https://en.wikipedia.org/wiki/Conditional_random_field) for an introduction to CRF.

In what follows we perform a very simple train/test splitting on the manually annotated letters of the [first volume of letters from Braun to Gehrard](http://arachne.uni-koeln.de/item/buch/2892) (1832-35). The original annotation is stored in [Webanno](http://nlp.dainst.org:18080/webanno/).

## Load the data

This is mainly done for the purpose of inspection. The Trainer will reload the data in its own constructor!

In [5]:
newcols = ["words", "pos", "lemma", "textlayer", "chunk", "entityid"]
corpus = KorrIOBCorpusReader("../data/IOB/", r".*\.iob", columntypes=newcols)

With the previous code we have load the data that are saved in one of the DAI's servers (nlp.dainst.org). The corpus includes the following total number of annotated pages:

In [6]:
len(corpus.fileids())

653

Here is one example of a file

In [7]:
corpus.fileids()[-1]

'3_Brunn1858_page241.iob'

And here are a few annotated words as read by our corpus reader:

In [8]:
corpus.full_tagged_words()[15:30]

[('1832', 'CARD', '@card@', 'HEAD', 'I-DATEletter'),
 ('Mein', 'PPOSAT', 'mein', '_', 'O'),
 ('freundlichster', 'ADJA', '<unknown>', '_', 'O'),
 ('Herr', 'NN', 'Herr', '_', 'O'),
 ('Professor', 'NN', 'Professor', '_', 'O'),
 ('Die', 'ART', 'die', '_', 'O'),
 ('Wohlthat', 'NE', '<unknown>', '_', 'O'),
 ('und', 'KON', 'und', '_', 'O'),
 ('Annehmlichkeit', 'NN', 'Annehmlichkeit', '_', 'O'),
 ('in', 'APPR', 'in', '_', 'O'),
 ('Ihrer', 'PPOSAT', 'ihr', '_', 'O'),
 ('nächsten', 'ADJA', 'nah', '_', 'O'),
 ('Nähe', 'NN', 'Nähe', '_', 'O'),
 ('zu', 'APPR', 'zu', '_', 'O'),
 ('logiren', 'NN', '<unknown>', '_', 'O')]

The length (in token number) of our corpus is:

In [9]:
len(corpus.words())

137197

# Training

In order to train our models, we make use of the `Trainer` class defined in the code of this project. The class should be *reasonably* transportable to other corpora of letters. We can discuss of any problems.

In [4]:
from training import Trainer

/Users/fmambrini/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/fmambrini/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [5]:
t = Trainer("../lib/config/korr_doc.json")

## Train and test split

How do you evaluate the performance of a trained model? Basically, the simplest answer is to split your manually annotated data in a `train` and a `test` part: the model trained on the 

If you want to learn more about this approach to the problem I suggest this very nice [introductory tutorial](https://www.youtube.com/watch?v=0pP4EwWJgIU&index=5&list=PL5-da3qGB5ICeMbQuqbbCOQWcS6OYBr5A).

CoNLL-like file structures have the peculiarity that they group the tokens into sentences, which must be kept together for the sake of linguistic coherence. Thus, methods of train-test splitting based on the randomized selection of tokens don't work. We have to come up with a custom solution.

(Unless, that is, you want to do someting like K-fold cross-validation. In this case, you might assume that any effect of distortion introduced by varying sentence lengths will be level out in the process of K-fold validation)

The Trainer object has a function that returns a train and a stesting split of the corpus using sentences as units. The counter will stop adding sentences to the `test` set after the last sentence break after the token counter has passed the select percentage (default: 20% test).

The following is the docstring of the method:

```python
def split(self, test_perc=0.2):
    """
    Split the corpus into training and testing. Randomize the selection of sentences and add sentences
    until the total number of tokens of the test section is equal or greater than test_perc.
    :param test_perc: percentage of tokens saved for tests
    :type test_perc: float (between 0 and 1)
    :return: tuple (of lists): train corpus, test corpus
    """
```

In [6]:
training,test = t.split()

As it can be seen, the training set is *roughly* equivalent to our given percentage (default = 80%). Given the mecahnism adopted to include sentences in the test set, the number of the training tokens will always be slightly inferior to selected one.

In [7]:
len([w for t in training for w in t]) / 137197

0.7999373164136242

Let us now assign these two sub-corpora to the `training` and `testing` properties of the Trainer

In [8]:
t.training = training
t.test = test

## Feature selections

In order to train a model you need three things:

* a $X$ matrix of features for your training data
* a $y$ vector of responses that correspond to these features
* a `template` that list what features must go on $X$ for each token in the training corpus

Optionally, if you have a test corpus, you want to extract $X$ and $y$ for your testing section as well, so that you will be able to use your trained model to the testing section too

The trainer has a `set_feats_labels` functions that take care to create both $X$ and $y$ for both training and testing corpora.

Training and testing corpus are taken from two properties of the Trainer, so no need to pass arguments. However, you'll need to import your template

In [9]:
from templates import template1

Before we proceed, `t.X_train, t.X_test, t.y_train` and `t.y_test` are all empty; e.g.

In [10]:
print(t.X_train is None)
print(t.X_train is None)
print(t.y_train is None)
print(t.y_test is None)

True
True
True
True


In [11]:
%%time
t.set_feats_labels(template1)

CPU times: user 20.4 s, sys: 523 ms, total: 20.9 s
Wall time: 21 s


What about now?

In [12]:
t.X_train[0][0]

{'BOS': 'True',
 'endsWithDigit[0]': 'False',
 'endsWithDigit[1]': 'False',
 'hasDigit[0]': 'False',
 'hasDigit[1]': 'False',
 'isDigit[0]': 'False',
 'isDigit[1]': 'False',
 'isInPersonDic[0]': 'False',
 'isInPersonDic[1]': 'False',
 'isInPersonDic[2]': 'False',
 'isInPlaceDic[0]': 'False',
 'isInPlaceDic[1]': 'True',
 'isInPlaceDic[2]': 'False',
 'isTitle[0]': 'True',
 'isTitle[1]': 'True',
 'isUpper[0]': 'False',
 'isUpper[1]': 'False',
 'lemma[0]': 'in',
 'lemma[1]': 'Rom',
 'pos[0]': 'APPR',
 'pos[0]|pos[1]': 'APPR|NE',
 'pos[0]|pos[1]|pos[2]': 'APPR|NE|VAFIN',
 'pos[1]': 'NE',
 'pos[1]|pos[2]': 'NE|VAFIN',
 'pos[2]': 'VAFIN',
 'prefix_long[0]': 'In',
 'prefix_long[1]': 'Rom',
 'prefix_long[2]': 'sind',
 'prefix_short[0]': 'In',
 'prefix_short[1]': 'Rom',
 'prefix_short[2]': 'sin',
 'rank[0]': '0',
 'sent_rank[0]': '0',
 'suffix_long[0]': 'In',
 'suffix_long[0]|suffix_long[1]': 'In|Rom',
 'suffix_long[0]|suffix_long[1]|suffix_long[2]': 'In|Rom|sind',
 'suffix_long[1]': 'Rom',
 'su

In [13]:
t.y_train[0][0]

'O'

Sanity check

In [15]:
assert len(t.y_train) == len(t.X_train)

In [17]:
assert len(t.y_test) == len(t.X_test)

## Model fitting

The last step is fitting the model!

In [18]:
%%time
t.fit()

CPU times: user 3min 21s, sys: 5.82 s, total: 3min 26s
Wall time: 3min 44s


# Evaluation

## Built-in `evaluate` function

This is good enough for some basic numbers, like e.g. the F1 score or precision and recall for each label

In [26]:
ev = t.evaluate()

/Users/fmambrini/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/fmambrini/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/fmambrini/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/fmambrini/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [29]:
print(ev["F1_class"])

                  precision    recall  f1-score   support

    B-DATEletter      0.945     0.939     0.942       147
    I-DATEletter      0.954     0.961     0.957       406
 B-DATEmentioned      0.889     0.545     0.676        44
 I-DATEmentioned      0.809     0.585     0.679        94
           B-LIT      0.862     0.543     0.667        46
           I-LIT      0.810     0.321     0.459        53
           B-OBJ      0.837     0.488     0.617        84
           I-OBJ      0.763     0.408     0.532        71
  B-ORGmentioned      0.932     0.775     0.846        71
  I-ORGmentioned      0.769     0.385     0.513        26
  B-PERaddressee      1.000     0.960     0.980       100
  I-PERaddressee      1.000     1.000     1.000         4
     B-PERauthor      0.981     0.946     0.963       111
     I-PERauthor      1.000     0.750     0.857         8
  B-PERmentioned      0.902     0.838     0.869       618
  I-PERmentioned      0.769     0.676     0.719       148
     B-PLACEf

The built-in evaluation is also reporting the 20 top-most likely transitions and unlikely transitions learned

In [31]:
for tr in ev["Top_likely_transitions"]: print(tr)

B-DATEletter -> I-DATEletter 6.017387
I-DATEmentioned -> I-DATEmentioned 5.336065
I-PERmentioned -> I-PERmentioned 5.334186
B-LIT  -> I-LIT   5.334048
B-DATEmentioned -> I-DATEmentioned 5.325046
B-ORGmentioned -> I-ORGmentioned 5.230235
I-ORGmentioned -> I-ORGmentioned 5.153614
I-DATEletter -> I-DATEletter 5.143415
B-OBJ  -> I-OBJ   5.114480
I-LIT  -> I-LIT   4.826870
B-PERmentioned -> I-PERmentioned 4.560642
B-PLACEmentioned -> I-PLACEmentioned 4.523414
I-OBJ  -> I-OBJ   4.502887
I-PLACEmentioned -> I-PLACEmentioned 4.214961
B-PLACEfrom -> I-PLACEfrom 3.726875
B-PERaddressee -> I-PERaddressee 3.489402
I-PLACEfrom -> I-PLACEfrom 3.179804
I-PERaddressee -> I-PERaddressee 2.955939
B-PERauthor -> I-PERauthor 2.679517
B-PERmentioned -> B-LIT   2.515660


In [32]:
for tr in ev["Top_unlikely_transitions"]: print(tr)

I-LIT  -> B-PERmentioned -1.127396
I-PERauthor -> O       -1.135229
I-DATEmentioned -> I-DATEletter -1.375594
B-PERmentioned -> I-LIT   -1.507606
B-PERauthor -> B-PERauthor -1.527896
I-DATEletter -> I-DATEmentioned -1.534933
B-PERmentioned -> I-OBJ   -1.538121
B-PERmentioned -> B-PERmentioned -1.592075
B-PERaddressee -> O       -1.744607
B-DATEletter -> O       -1.923546
O      -> I-PLACEfrom -1.974725
O      -> I-PERauthor -2.034647
O      -> I-PERaddressee -2.428154
O      -> I-PLACEmentioned -2.897769
O      -> I-DATEletter -3.279177
O      -> I-DATEmentioned -3.385009
O      -> I-ORGmentioned -3.629834
O      -> I-LIT   -4.995137
O      -> I-OBJ   -5.420429
O      -> I-PERmentioned -5.687708


## Evaluation with `eli5`

The module eli5 provides some nicer graphic and functions to inspect what are the most highly rewarded features in the learning process

In [33]:
import eli5

In [40]:
#eli5.show_weights(crf, top=10, targets=['O', 'B-ORG', 'I-ORG'])
eli5.show_weights(t.crf, targets=['O','B-PLACEmentioned', "I-PLACEmentioned"])

In [52]:
eli5.show_weights(t.crf, top=10, feature_re='^lemma',
                  horizontal_layout=False, show=['targets'])

# Final model

The final model is trained with the totality of the corpus.

In [41]:
tfinal = Trainer("../lib/config/korr_doc.json")

In [45]:
assert len(tfinal._corpus.sents()) == len(tfinal.training)

In [46]:
%%time
tfinal.set_feats_labels(template1)

CPU times: user 26 s, sys: 1.19 s, total: 27.2 s
Wall time: 46.4 s


In [47]:
%%time
tfinal.fit()

CPU times: user 4min 9s, sys: 8.87 s, total: 4min 18s
Wall time: 4min 31s


Now, let's write everything down

In [48]:
import pickle
pickle.dump( tfinal.crf, open( "../lib/models/korrespondez_model_stage3.pickle", "wb" ) )

(let's verify that everything went smoothly)

In [53]:
with open("../lib/models/korrespondez_model_stage3.pickle", "rb") as f:
    crf = pickle.load(f)

In [55]:
crf.predict(t.X_test)[0]

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

# Summary

All the previous operations in one cell

## With training/test splitting

Use the following cell if you want to do train/test splitting. Adjust the test percentage and the path to the config as you see fit

In [ ]:
from templates import template1

t = Trainer("../lib/config/korr_doc.json")
train,test = t.split()
t.train = train
t.test = test

t.set_feats_labels(template1)
t.fit()

ev = t.evaluate()

In [ ]:
#inspect ev for the evaluation
print(ev["F1_general"])
print(ev["F1_class"])

## Train with the full corpus

Use the following cell if you want to use the full corpus for training

In [ ]:
from templates import template1
import pickle

t = Trainer("../lib/config/korr_doc.json")
t.set_feats_labels(template1)
t.fit()

Use the following cell to write everything to a pickle file.

Don't forget to **update the output file name**!

In [ ]:
outpath = 
with open(outpath, 'w') as out:
    pickle.dump( t.crf, out)